In [1]:
import pandas as pd
import json
import requests
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 12]

In [2]:
import networkx as nx

from networkx.readwrite import json_graph
from networkx.algorithms import community

In [3]:
df = pd.read_csv('2008.csv')[['Month', 'DayofMonth', 'Year', 'ArrTime', 'Origin', 'Dest', 'Cancelled', 'FlightNum']]

In [4]:
df['Date'] = df['DayofMonth'].astype(str) +"."+ df['Month'].astype(str) +"."+ df['Year'].astype(str)

In [5]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=False)

In [6]:
df.head()

,Month,DayofMonth,Year,ArrTime,Origin,Dest,Cancelled,FlightNum,Date
0,1,3,2008,2211.0,IAD,TPA,0,335,2008-03-01
1,1,3,2008,1002.0,IAD,TPA,0,3231,2008-03-01
2,1,3,2008,804.0,IND,BWI,0,448,2008-03-01
3,1,3,2008,1054.0,IND,BWI,0,1746,2008-03-01
4,1,3,2008,1959.0,IND,BWI,0,3920,2008-03-01


In [7]:
df = df.loc[df['Cancelled'] != 1]
df = df.drop(['Month', 'DayofMonth', 'Year'], axis=1)

In [8]:
df.head()

,ArrTime,Origin,Dest,Cancelled,FlightNum,Date
0,2211.0,IAD,TPA,0,335,2008-03-01
1,1002.0,IAD,TPA,0,3231,2008-03-01
2,804.0,IND,BWI,0,448,2008-03-01
3,1054.0,IND,BWI,0,1746,2008-03-01
4,1959.0,IND,BWI,0,3920,2008-03-01


In [9]:
df = df.drop(['Cancelled'], axis=1)

In [10]:
df = df.sort_values(by=['ArrTime', 'Date'])

In [11]:
df

,ArrTime,Origin,Dest,FlightNum,Date
126170,1.0,CLE,DAY,2104,2008-01-01
229187,1.0,DEN,ICT,390,2008-01-01
254540,1.0,IAD,MCO,1569,2008-01-01
296470,1.0,LAS,PVD,416,2008-01-01
486109,1.0,SJC,AUS,502,2008-01-01
...,...,...,...,...,...
6874373,NaN,LGA,GRR,2151,2008-12-31
6919273,NaN,DFW,SNA,1233,2008-12-31
6930013,NaN,ORD,SNA,1677,2008-12-31
6954522,NaN,SEA,SNA,516,2008-12-31


In [16]:
import random
from tqdm import tqdm

airports = pd.concat([df['Origin'], df['Dest']])
airports = pd.Series(airports.unique())
spread_infection = pd.DataFrame(airports, columns=['airport'])
spread_infection['infection'] = 0
spread_infection.loc[spread_infection['airport'] == airport, 'infection'] = 1

def start_infection(airport, prob, df):
    
    all_infested_iters = 0
    
    for i in tqdm(df.itertuples()):
        
        airport_of_origin = i[2]
        airport_of_dest = i[3]
        infected_or_not_orig = spread_infection.loc[spread_infection['airport'] == airport_of_origin, 'infection']
        infected_or_not_dest = spread_infection.loc[spread_infection['airport'] == airport_of_dest, 'infection']
        
        if (infected_or_not_dest == 1).all():
            continue # нас не интересуют рейсы в заразные города
        
        if (spread_infection['infection'] == 1).all():
            print('Evil is WON!!!')
            return all_infested_iters
        
        if (infected_or_not_orig == 1).all() and (infected_or_not_dest != 1).all():
            if random.random() <= prob:
                spread_infection.loc[spread_infection['airport'] == airport_of_dest, 'infection'] = 1
        
        all_infested_iters += 1

In [ ]:
start_infection('FAR', 1, df)